###  $TensorboardX$

```python
pip install tensorboardX
```

### $Visdom$

```python
python -m visdom.server
```

$lines:single\space trace$

```python
from visdom import Visdom
viz = Visdom()
viz.line([0.],[0.],win = 'train_loss',opts = dict(title = 'train loss'))
viz.line([loss.item()],[global_step],win = 'train_loss',update = 'append')
```

$lines:multi-traces$
```python
from visdom import Visdom
viz = Visdom()
viz.line([[0.0,0.0]],[0.],win = 'test',opts = dict(title = 'test loss & acc',
                                   legend = ['loss','acc']))
viz.line([[test_loss,correct / len(test_loader.dataset)]],
         [global_step],win = 'test',update = 'append')
```

$visual\space X$

```python
viz = Visdom()
viz.images(data.view(-1,1,28,28),win = 'x')
viz.text(str(pred.detach().cpu().numpy()),win = 'pred',opts = dict(title = 'pred'))
```

In [1]:
import torch
import torch.nn as nn
from torchvision import datasets,transforms
import torch.optim as optim
from torch.nn import functional as F
from visdom import Visdom
import torchvision

In [2]:
batch_size = 64
learning_rate  = 1e-2
epochs = 10

In [3]:
train_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('datasets/mnist_data',
                train=True,
                download=True,
                transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),                       # 数据类型转化
                torchvision.transforms.Normalize((0.1307, ), (0.3081, )) # 数据归一化处理
    ])), batch_size=batch_size,shuffle=True)

test_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('datasets/mnist_data/',
                train=False,
                download=True,
                transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307, ), (0.3081, ))
    ])),batch_size=batch_size,shuffle=False)

In [4]:
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)

        return x

In [5]:
device = torch.device('cuda:0')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(),lr = learning_rate)
criteon = nn.CrossEntropyLoss().to(device)

viz = Visdom()

viz.line([0.], [0.], win='train_loss', opts=dict(title='train loss'))
viz.line([[0.0, 0.0]], [0.], win='test', opts=dict(title='test loss&acc.',
                                                   legend=['loss', 'acc.']))
global_step = 0

Setting up a new session...


In [6]:
for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        data, target = data.to(device), target.cuda()

        logits = net(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        global_step += 1
        viz.line([loss.item()], [global_step], win='train_loss', update='append')

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.argmax(dim=1)
        correct += pred.eq(target).float().sum().item()

    viz.line([[test_loss, correct / len(test_loader.dataset)]],
             [global_step], win='test', update='append')
    viz.images(data.view(-1, 1, 28, 28), win='x')
    viz.text(str(pred.detach().cpu().numpy()), win='pred',
             opts=dict(title='pred'))

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.293349
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.084863
Train Epoch: 0 [12800/60000 (21%)]	Loss: 1.405478
Train Epoch: 0 [19200/60000 (32%)]	Loss: 1.005854
Train Epoch: 0 [25600/60000 (43%)]	Loss: 1.209227
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.782526
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.590590
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.607945
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.300273
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.343110

Test set: Average loss: 0.0058, Accuracy: 8984.0/10000 (90%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.525279
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.288890
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.481225
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.351091
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.215934
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.545303
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.232588
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.169912
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.143733